# Predict Iron Ore Impurities (% of Silica) with ONNX in SQL Edge

## Import the required packages for this experiment to run. This script has been tested against the following packages versions

azureml.core - 1.16.0.post1
azureml.automl.core  1.16.0
azureml.automl.runtime  1.16.0
azureml.train.automl 1.16.0
matplotlib - 3.3.1
numpy -  1.16.6
pandas - 0.23.4
pyodbc - 4.0.30
spicy - 1.4.1

In [ ]:
import logging
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pyodbc

from scipy import stats
from scipy.stats import skew #for some statistics

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl import constants

## Update the Subscription ID, Resource Group Name and the ML workspace name in the cell below


In [ ]:
ws = Workspace(subscription_id="<Your_Azure_Subscription_ID>",
               resource_group="<Your_Resource_Group_Name",
               workspace_name="<Your_Azure_ML_Workspace_Name>")
# Choose a name for the experiment.
experiment_name = 'silic_percent2-Forecasting-onnx'
experiment = Experiment(ws, experiment_name)

## Import the Data Set into a panda frame. For the purpose of the model training we are using a variation of the MiningProcess_Flotation_Plant_Database.csv file available [here](https://www.kaggle.com/edumagalhaes/quality-prediction-in-a-mining-process) from Kaggle. 
Update the path to the CSV file.te

In [ ]:
df = pd.read_csv("<Path To the CSV File>",decimal=",",parse_dates=["date"],infer_datetime_format=True)
df = df.drop(['date'],axis=1)
df.describe()

## Analyze the data to identiy any skewness in the data. During this process we will look at the distribution and the skew information for each of the columns in the data frame. 

In [ ]:
## We can use a histogram chart to view the data distribution for the Dataset. In this example, we are looking at the histogram for the "% Silica Concentrate" 
## and the "% Iron Feed". From the histogram, you'll notice the data distribution is skewed for most of the features in the dataset. 

f, (ax1,ax2,ax3) = plt.subplots(1,3)
ax1.hist(df['% Iron Feed'], bins='auto')
#ax1.title = 'Iron Feed'
ax2.hist(df['% Silica Concentrate'], bins='auto')
#ax2.title = 'Silica Concentrate'
ax3.hist(df['% Silica Feed'], bins='auto')
#ax3.title = 'Silica Feed'


## Check and fix the level of skweness in the data. 

In [ ]:
##Check data skewness with the skew or the kurtosis function in spicy.stats
##Skewness using the spicy.stats skew function
for i in list(df):
        print('Skew value for column "{0}" is: {1}'.format(i,skew(df[i])))

In [ ]:
#Fix the Skew using Box Cox Transform
from scipy.special import boxcox1p
for i in list(df):
    if(abs(skew(df[i])) >= 0.20):
        #print('found skew in column - {0}'.format(i))
        df[i] = boxcox1p(df[i], 0.10)
        print('Skew value for column "{0}" is: {1}'.format(i,skew(df[i])))

### Check the correlation of other features with the prediction feature. If the correlation is not high, remove those features.

In [ ]:
silic_corr = df.corr()['% Silica Concentrate']
silic_corr = abs(silic_corr).sort_values()
drop_index= silic_corr.index[:8].tolist()
df = df.drop(drop_index, axis=1)
df.rename(columns={'Flotation Column 07 Air Flow':'Flotation Column 04 Air Flow'},inplace=True)
df.rename(columns={'Flotation Column 04 Level':'Flotation Column 01 Level'},inplace=True)
df.rename(columns={'Flotation Column 05 Level':'Flotation Column 02 Level'},inplace=True)
df.rename(columns={'Flotation Column 06 Level':'Flotation Column 03 Level'},inplace=True)
df.rename(columns={'Flotation Column 07 Level':'Flotation Column 04 Level'},inplace=True)
df.describe()

### Start the AzureML Experiment to find and train the best algorithem. In this case, we are testing with all regression algorithms, with a primary metric of Normalized Root Mean Squared Error (NRMSE). For more information refer [Azure ML Experiments Primary Metric](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric).

## The code below will start a local run of the ML experiment. 

In [ ]:
## Define the X_train and the y_train data sets for the AutoML experiments. X_Train are the inputs or the features, while y_train is the outcome or the prediction result. 

y_train = df['% Silica Concentrate']
x_train = df.iloc[:,0:-1]
automl_config = AutoMLConfig(task = 'regression',
                             primary_metric = 'normalized_root_mean_squared_error',
                             iteration_timeout_minutes = 60,
                             iterations = 10,                        
                             X = x_train, 
                             y = y_train,
                             featurization = 'off',
                             enable_onnx_compatible_models=True)



In [ ]:
local_run = experiment.submit(automl_config, show_output = True)
best_run, onnx_mdl = local_run.get_output(return_onnx_model=True)

## Load the Model in SQL Database Edge database for local scoring. 
### Make sure to change the SQL Edge instance IP (in the format IP,PortNumber or name,PortNumber) and Password details

In [ ]:
## Load the Model into a SQL Database.
## Define the Connection string parameters. These connection strings will be used later also in the demo.
server = '' # SQL Edge IP, Port address
username = 'sa' # SQL Server username
password = '' # SQL Server password
database = 'IronOreSilicaPrediction'
db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"
conn = pyodbc.connect(db_connection_string, autocommit=True)
cursor = conn.cursor()

# Insert the ONNX model into the models table
query = f"insert into models ([description], [data]) values ('Silica_Percentage_Predict_Regression_NRMSE_New1',?)"
model_bits = onnx_mdl.SerializeToString()
insert_params  = (pyodbc.Binary(model_bits))
cursor.execute(query, insert_params)
conn.commit()
cursor.close()
conn.close()

## Use the SQL Database Edge Model to perform perdictions using the trained model.
### Make sure to change the SQL Edge instance IP (in the format IP,PortNumber or name,PortNumber) and Password details

In [ ]:
## Define the Connection string parameters. These connection strings will be used later also in the demo.
server = '' # SQL Edge IP, Port address
username = 'sa' # SQL Server username
password = '' # SQL Server password
database = 'IronOreSilicaPrediction'
db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"
conn = pyodbc.connect(db_connection_string, autocommit=True)
#cursor = conn.cursor()
query = \
        f'declare @model varbinary(max) = (Select [data] from [dbo].[Models] where [id] = 1);' \
        f' with d as ( SELECT  [timestamp] ,cast([cur_Iron_Feed] as real) [__Iron_Feed] ,cast([cur_Silica_Feed]  as real) [__Silica_Feed]' \
        f',cast([cur_Starch_Flow] as real) [Starch_Flow],cast([cur_Amina_Flow] as real) [Amina_Flow]' \
        f' ,cast([cur_Ore_Pulp_pH] as real) [Ore_Pulp_pH] ,cast([cur_Flotation_Column_01_Air_Flow] as real) [Flotation_Column_01_Air_Flow]' \
        f' ,cast([cur_Flotation_Column_02_Air_Flow] as real) [Flotation_Column_02_Air_Flow]' \
        f' ,cast([cur_Flotation_Column_03_Air_Flow] as real) [Flotation_Column_03_Air_Flow]' \
        f' ,cast([cur_Flotation_Column_04_Air_Flow] as real) [Flotation_Column_04_Air_Flow]' \
        f' ,cast([cur_Flotation_Column_01_Level] as real) [Flotation_Column_01_Level]' \
        f' ,cast([cur_Flotation_Column_02_Level] as real) [Flotation_Column_02_Level]' \
        f' ,cast([cur_Flotation_Column_03_Level] as real) [Flotation_Column_03_Level]' \
        f' ,cast([cur_Flotation_Column_04_Level] as real) [Flotation_Column_04_Level]' \
        f' ,cast([cur_Iron_Concentrate] as real) [__Iron_Concentrate]' \
        f' FROM [dbo].[IronOreMeasurements]' \
        f' where timestamp between dateadd(minute,-10,getdate()) and getdate()) ' \
        f' SELECT d.*, p.variable_out1' \
        f' FROM PREDICT(MODEL = @model, DATA = d, Runtime=ONNX) WITH(variable_out1 numeric(25,17)) as p;' 
  
df_result = pd.read_sql(query,conn)
df_result.describe()



## Use Python to create a chart of the Predcited Silica Percentage against the Iron Feed, Datetime, Silica Feed.

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_result['timestamp'],y=df_result['__Iron_Feed'],mode='lines+markers',name='Iron Feed',line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x=df_result['timestamp'],y=df_result['__Silica_Feed'],mode='lines+markers',name='Silica Feed',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=df_result['timestamp'],y=df_result['variable_out1'],mode='lines+markers',name='Silica Percent',line=dict(color='royalblue', width=3)))
fig.update_layout(height= 600, width=1500,xaxis_title='Time')
fig.show()